In [4]:
import asyncio
import pandas as pd
from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
from datetime import datetime, timedelta
from functions import marketId_to_ticker

# Initialize the network and client
network = Network.local()
client = AsyncClient(network)

async def fetch_derivative_orders(subaccount_id, market_ids, start_date, end_date, client):
    # Convert start and end dates to milliseconds since epoch
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)

    orders_list = []
    skip = 0
    limit = 100  # Set the number of results to fetch per request

    while True:
        pagination = PaginationOption(skip=skip, limit=limit)

        # Fetch the orders history
        orders_response = await client.fetch_derivative_orders_history(
            subaccount_id=subaccount_id,
            market_ids=market_ids,
            pagination=pagination,
        )

        # Break the loop if no orders were returned
        if not orders_response['orders']:
            break

        # Iterate through each order
        for order in orders_response['orders']:
            created_at = pd.to_datetime(order['createdAt'], unit='ms')
            updated_at = pd.to_datetime(order['updatedAt'], unit='ms')

            if created_at < pd.to_datetime(start_timestamp, unit='ms'):
                return orders_list

            orders_list.append({
                'orderHash': order['orderHash'],
                'marketId': order['marketId'],
                'subaccountId': order['subaccountId'],
                'executionType': order['executionType'],
                'orderType': order['orderType'],
                'price': order['price'],
                'quantity': order['quantity'],
                'updated_at': updated_at,
                'state': order['state'],
                'created_at': created_at,
            })

        # Increment skip for pagination
        skip += limit

    return orders_list

async def main():
    # List of subaccount IDs

    subaccount_id = "0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000048" 
    ticker = "iTSLA/USDT PERP"
    # Market IDs for fetching orders
 

    now = datetime.now()
    start_date = (now - timedelta(hours=24)).strftime('%Y-%m-%d %H:%M:%S')
    end_date = now.strftime('%Y-%m-%d %H:%M:%S')


    market_id = await marketId_to_ticker(ticker)
    # Fetch orders for each subaccount
    orders = await fetch_derivative_orders(subaccount_id, market_id, start_date, end_date, client)

    # Convert all orders to a DataFrame and save as a single CSV
    df_all_orders = pd.DataFrame(orders)
    csv_name = "orders_c4z.csv"
    df_all_orders.to_csv(csv_name, index=False)
    print(f"All orders saved to {csv_name}")

await main()

All orders saved to orders_c4z.csv
